In [1]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.1 MB/s eta 0:00:00


In [2]:
import optuna

In [122]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score , mean_squared_error
import xgboost as xgb

In [5]:
df = pd.read_csv('Admission_Predict.csv')

In [6]:
df.head(5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         400 non-null    int64  
 1   GRE Score          400 non-null    int64  
 2   TOEFL Score        400 non-null    int64  
 3   University Rating  400 non-null    int64  
 4   SOP                400 non-null    float64
 5   LOR                400 non-null    float64
 6   CGPA               400 non-null    float64
 7   Research           400 non-null    int64  
 8   Chance of Admit    400 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 28.2 KB


In [8]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [9]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [10]:
X = df.drop(['Serial No.' , 'Chance of Admit '] , axis = 1)

In [11]:
y = df['Chance of Admit ']

In [12]:
X

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
395,324,110,3,3.5,3.5,9.04,1
396,325,107,3,3.0,3.5,9.11,1
397,330,116,4,5.0,4.5,9.45,1
398,312,103,3,3.5,4.0,8.78,0


In [13]:
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
395    0.82
396    0.84
397    0.91
398    0.67
399    0.95
Name: Chance of Admit , Length: 400, dtype: float64

In [14]:
# train test split

X_train , X_test , y_train , y_test = train_test_split(X, y , test_size = 0.20 , random_state=30)

In [15]:
X_train

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
313,301,100,3,3.5,3.0,8.04,0
21,325,114,4,3.0,2.0,8.40,0
222,324,113,4,4.5,4.0,8.79,0
350,318,107,3,3.0,3.5,8.27,1
398,312,103,3,3.5,4.0,8.78,0
...,...,...,...,...,...,...,...
309,308,110,4,3.5,3.0,8.60,0
386,302,101,2,2.5,3.5,7.96,0
140,329,110,2,4.0,3.0,9.15,1
301,319,108,2,2.5,3.0,8.76,0


In [16]:
y_train

313    0.67
21     0.70
222    0.76
350    0.74
398    0.67
       ... 
309    0.70
386    0.46
140    0.84
301    0.66
293    0.64
Name: Chance of Admit , Length: 320, dtype: float64

In [17]:
std_sca = StandardScaler()

In [18]:
X_train = std_sca.fit_transform(X_train)
X_test = std_sca.transform(X_test)

In [19]:
X_train

array([[-1.38445057, -1.20302748, -0.0436852 , ..., -0.47668654,
        -0.92722233, -1.057925  ],
       [ 0.74291645,  1.11606164,  0.83001885, ..., -1.57805616,
        -0.32227238, -1.057925  ],
       [ 0.65427616,  0.95041241,  0.83001885, ...,  0.62468308,
         0.33309006, -1.057925  ],
       ...,
       [ 1.09747762,  0.45346475, -0.91738926, ..., -0.47668654,
         0.93804002,  0.94524659],
       [ 0.2110747 ,  0.1221663 , -0.91738926, ..., -0.47668654,
         0.28267757, -1.057925  ],
       [-0.40940735, -1.53432592, -1.79109332, ..., -0.47668654,
        -0.69196402,  0.94524659]])

In [20]:
X_test

array([[ 0.29971499,  0.45346475,  1.70372291,  1.59677891,  1.7260527 ,
         1.02206084,  0.94524659],
       [-1.65037144, -1.03737826, -0.91738926, -1.82598163, -1.57805616,
        -1.22969731, -1.057925  ],
       [ 0.03379411,  0.45346475, -0.0436852 ,  0.61884733,  1.17536789,
         0.87082335,  0.94524659],
       [-0.23212677, -0.20913214, -0.0436852 , -0.35908425,  1.7260527 ,
         0.51793588, -1.057925  ],
       [-0.58668793, -0.04348292, -0.0436852 ,  0.12988154,  0.07399827,
         0.13144008, -1.057925  ],
       [ 1.3633985 ,  1.77865853,  1.70372291,  1.59677891,  1.7260527 ,
         1.4757733 ,  0.94524659],
       [ 0.83155674,  0.61911397,  1.70372291,  1.10781312,  0.62468308,
         1.07247334,  0.94524659],
       [ 1.2747582 ,  1.94430775,  0.83001885,  1.59677891,  1.17536789,
         1.25731916,  0.94524659],
       [-0.32076706, -0.70607981, -0.0436852 ,  0.61884733,  0.62468308,
         0.2658734 , -1.057925  ],
       [-0.40940735,  0.2878

In [21]:
def objective(trail,data=X,target=y):
  train_x,test_x,train_y,test_y=train_test_split(data,target,test_size=0.25,random_state=30)
  param={
      "tree_method":"gpu_hist",
      'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
      'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
      'colsample_bytree' :trail.suggest_categorical('colsample_bytree' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'subsample' :trail.suggest_categorical('subsample' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
      'n_estimators' :3000,
      'max_depth' :trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
      'random_state' :trail.suggest_categorical('random_state' , [10,20,30 ,2000 , 3454,243123]),
      'min_child_weight' :trail.suggest_int('min_child_weight' ,1,200)
      }
  Xgb_reg_model=xgb.XGBRegressor(**param)
  Xgb_reg_model.fit(train_x,train_y,eval_set=[(test_x,test_y)],verbose=True)
  pred_xgb=Xgb_reg_model.predict(test_x)

In [22]:
find_params=optuna.create_study()
find_params.optimize(objective,n_trials=5)
find_params.best_trial.params

[I 2023-05-28 14:29:22,854] A new study created in memory with name: no-name-4b3e23a1-f1e5-4433-b916-0c0cddfc0bce
<ipython-input-21-8fe9b5c35c92>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
<ipython-input-21-8fe9b5c35c92>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),


[0]	validation_0-rmse:0.26380
[1]	validation_0-rmse:0.26380
[2]	validation_0-rmse:0.26379
[3]	validation_0-rmse:0.26379
[4]	validation_0-rmse:0.26379
[5]	validation_0-rmse:0.26379
[6]	validation_0-rmse:0.26379
[7]	validation_0-rmse:0.26378
[8]	validation_0-rmse:0.26378
[9]	validation_0-rmse:0.26378
[10]	validation_0-rmse:0.26378
[11]	validation_0-rmse:0.26378
[12]	validation_0-rmse:0.26377
[13]	validation_0-rmse:0.26377
[14]	validation_0-rmse:0.26377
[15]	validation_0-rmse:0.26377
[16]	validation_0-rmse:0.26377
[17]	validation_0-rmse:0.26376
[18]	validation_0-rmse:0.26376
[19]	validation_0-rmse:0.26376
[20]	validation_0-rmse:0.26376
[21]	validation_0-rmse:0.26376
[22]	validation_0-rmse:0.26375
[23]	validation_0-rmse:0.26375
[24]	validation_0-rmse:0.26375
[25]	validation_0-rmse:0.26375
[26]	validation_0-rmse:0.26374
[27]	validation_0-rmse:0.26374
[28]	validation_0-rmse:0.26374
[29]	validation_0-rmse:0.26374
[30]	validation_0-rmse:0.26374
[31]	validation_0-rmse:0.26373
[32]	validation_0-

[W 2023-05-28 14:29:35,104] Trial 0 failed with parameters: {'lambda': 5.635321299456835, 'alpha': 2.0226419526917705, 'colsample_bytree': 0.6, 'subsample': 1, 'learning_rate': 1e-05, 'max_depth': 12, 'random_state': 3454, 'min_child_weight': 80} because of the following error: The value None could not be cast to float..
[W 2023-05-28 14:29:35,112] Trial 0 failed with value None.


[0]	validation_0-rmse:0.10237
[1]	validation_0-rmse:0.09991
[2]	validation_0-rmse:0.09919
[3]	validation_0-rmse:0.09919
[4]	validation_0-rmse:0.09919
[5]	validation_0-rmse:0.09919
[6]	validation_0-rmse:0.09919
[7]	validation_0-rmse:0.09919
[8]	validation_0-rmse:0.09771
[9]	validation_0-rmse:0.09771
[10]	validation_0-rmse:0.09771
[11]	validation_0-rmse:0.09771
[12]	validation_0-rmse:0.09727
[13]	validation_0-rmse:0.09727
[14]	validation_0-rmse:0.09727
[15]	validation_0-rmse:0.09727
[16]	validation_0-rmse:0.09727
[17]	validation_0-rmse:0.09727
[18]	validation_0-rmse:0.09727
[19]	validation_0-rmse:0.09727
[20]	validation_0-rmse:0.09727
[21]	validation_0-rmse:0.09727
[22]	validation_0-rmse:0.09727
[23]	validation_0-rmse:0.09727
[24]	validation_0-rmse:0.09727
[25]	validation_0-rmse:0.09727
[26]	validation_0-rmse:0.09727
[27]	validation_0-rmse:0.09727
[28]	validation_0-rmse:0.09727
[29]	validation_0-rmse:0.09727
[30]	validation_0-rmse:0.09727
[31]	validation_0-rmse:0.09727
[32]	validation_0-

[W 2023-05-28 14:29:44,845] Trial 1 failed with parameters: {'lambda': 0.00191380140081088, 'alpha': 0.08395335747483855, 'colsample_bytree': 0.6, 'subsample': 1, 'learning_rate': 1, 'max_depth': 4, 'random_state': 10, 'min_child_weight': 110} because of the following error: The value None could not be cast to float..
[W 2023-05-28 14:29:44,846] Trial 1 failed with value None.


...

[0]	validation_0-rmse:0.14556
[1]	validation_0-rmse:0.14556
[2]	validation_0-rmse:0.14556
[3]	validation_0-rmse:0.14556
[4]	validation_0-rmse:0.14556
[5]	validation_0-rmse:0.14556
[6]	validation_0-rmse:0.14551
[7]	validation_0-rmse:0.14551
[8]	validation_0-rmse:0.14544
[9]	validation_0-rmse:0.14542
[10]	validation_0-rmse:0.14542
[11]	validation_0-rmse:0.14548
[12]	validation_0-rmse:0.14548
[13]	validation_0-rmse:0.14548
[14]	validation_0-rmse:0.14548
[15]	validation_0-rmse:0.14554
[16]	validation_0-rmse:0.14554
[17]	validation_0-rmse:0.14554
[18]	validation_0-rmse:0.14554
[19]	validation_0-rmse:0.14554
[20]	validation_0-rmse:0.14554
[21]	validation_0-rmse:0.14550
[22]	validation_0-rmse:0.14550
[23]	validation_0-rmse:0.14550
[24]	validation_0-rmse:0.14548
[25]	validation_0-rmse:0.14548
[26]	validation_0-rmse:0.14548
[27]	validation_0-rmse:0.14548
[28]	validation_0-rmse:0.14548
[29]	validation_0-rmse:0.14548
[30]	validation_0-rmse:0.14548
[31]	validation_0-rmse:0.14548
[32]	validation_0-

[W 2023-05-28 14:30:05,095] Trial 3 failed with parameters: {'lambda': 0.00028494942490322344, 'alpha': 1.3855852455210516, 'colsample_bytree': 0.2, 'subsample': 0.5, 'learning_rate': 1, 'max_depth': 11, 'random_state': 2000, 'min_child_weight': 101} because of the following error: The value None could not be cast to float..
[W 2023-05-28 14:30:05,101] Trial 3 failed with value None.


[0]	validation_0-rmse:0.26380
[1]	validation_0-rmse:0.26380
[2]	validation_0-rmse:0.26380
[3]	validation_0-rmse:0.26380
[4]	validation_0-rmse:0.26380
[5]	validation_0-rmse:0.26380
[6]	validation_0-rmse:0.26380
[7]	validation_0-rmse:0.26380
[8]	validation_0-rmse:0.26380
[9]	validation_0-rmse:0.26380
[10]	validation_0-rmse:0.26380
[11]	validation_0-rmse:0.26380
[12]	validation_0-rmse:0.26380
[13]	validation_0-rmse:0.26380
[14]	validation_0-rmse:0.26380
[15]	validation_0-rmse:0.26380
[16]	validation_0-rmse:0.26380
[17]	validation_0-rmse:0.26380
[18]	validation_0-rmse:0.26380
[19]	validation_0-rmse:0.26380
[20]	validation_0-rmse:0.26380
[21]	validation_0-rmse:0.26380
[22]	validation_0-rmse:0.26380
[23]	validation_0-rmse:0.26380
[24]	validation_0-rmse:0.26380
[25]	validation_0-rmse:0.26380
[26]	validation_0-rmse:0.26380
[27]	validation_0-rmse:0.26380
[28]	validation_0-rmse:0.26380
[29]	validation_0-rmse:0.26380
[30]	validation_0-rmse:0.26380
[31]	validation_0-rmse:0.26380
[32]	validation_0-

[W 2023-05-28 14:30:14,645] Trial 4 failed with parameters: {'lambda': 0.0055876480465506155, 'alpha': 0.00014946167574209491, 'colsample_bytree': 0.7, 'subsample': 0.2, 'learning_rate': 8, 'max_depth': 5, 'random_state': 30, 'min_child_weight': 199} because of the following error: The value None could not be cast to float..
[W 2023-05-28 14:30:14,650] Trial 4 failed with value None.


ValueError: ignored

In [23]:
find_params.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,None,2023-05-28 14:29:22.856602,2023-05-28 14:29:35.104436,0 days 00:00:12.247834,2.022642,0.6,5.635321,0.00001,12,80,3454,1.0,FAIL
1,1,None,2023-05-28 14:29:35.114078,2023-05-28 14:29:44.844882,0 days 00:00:09.730804,0.083953,0.6,0.001914,1.00000,4,110,10,1.0,FAIL
2,2,None,2023-05-28 14:29:44.848583,2023-05-28 14:29:55.044402,0 days 00:00:10.195819,0.003144,0.1,0.004442,1.00000,6,134,2000,0.4,FAIL
3,3,None,2023-05-28 14:29:55.053416,2023-05-28 14:30:05.095580,0 days 00:00:10.042164,1.385585,0.2,0.000285,1.00000,11,101,2000,0.5,FAIL
4,4,None,2023-05-28 14:30:05.103506,2023-05-28 14:30:14.645585,0 days 00:00:09.542079,0.000149,0.7,0.005588,8.00000,5,199,30,0.2,FAIL


In [24]:
optuna.visualization.plot_optimization_history(find_params)

[W 2023-05-28 14:30:59,564] There are no complete trials.


In [25]:
optuna.visualization.plot_slice(find_params)

[W 2023-05-28 14:31:06,163] Your study does not have any completed trials.


In [26]:
optuna.visualization.plot_contour(find_params,params=['alpha','lambda'])



[W 2023-05-28 14:31:11,047] Your study does not have any completed trials.


In [27]:
best_params={'lambda': 0.7816338595163784,
 'alpha': 0.25683131748959687,
 'colsample_bytree': 0.1,
 'subsample': 0.9,
 'learning_rate': 1,
 'max_depth': 10,
 'random_state': 20,
 'min_child_weight': 124}

In [28]:
model=xgb.XGBRegressor(**best_params)

In [29]:
model.fit(X_train , y_train)

XGBRegressor(alpha=0.25683131748959687, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.1, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.7816338595163784,
             learning_rate=1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
             max_leaves=None, min_child_weight=124, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [30]:
y_pred = model.predict(X_test)

In [31]:
r2score=[0,1]
from sklearn.metrics import r2_score
r2_score(y_pred , y_test)

0.5163766154149774

In [32]:
from sklearn.ensemble import RandomForestRegressor
model2=RandomForestRegressor()
model2.fit(X_train,y_train)
y_pred2=model2.predict(X_test)
r2_score(y_test,y_pred2)


0.7930767166621849

In [33]:
# classification problem statemnet


In [76]:
# tipes dataset import 
df = sns.load_dataset('tips') 

In [77]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [79]:
df.isna().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [80]:
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()
df['time']=encode.fit_transform(df["time"])

In [81]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    int64   
 6   size        244 non-null    int64   
dtypes: category(3), float64(2), int64(2)
memory usage: 8.9 KB


In [85]:
X = df.drop('time' , axis = 1)
y = df['time']

In [86]:
X

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [87]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int64

In [96]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include= 'category').columns
numerical_cols = X.select_dtypes(exclude="category").columns

In [97]:
categorical_cols

Index(['sex', 'smoker', 'day'], dtype='object')

In [98]:
numerical_cols

Index(['total_bill', 'tip', 'size'], dtype='object')

In [89]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OneHotEncoder # OnehotEncoder
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [103]:
#numerical_columns

num_pipeline = Pipeline(
                steps=[
                      ('Imputer' , SimpleImputer(strategy='median')),
                      ('scaler' , StandardScaler())
                      
                    
                ])

cat_pipeline =Pipeline(
                steps =[
                    ('Imputer' , SimpleImputer(strategy = 'most_frequent')),
                    ('one_hot_encoder' , OneHotEncoder(sparse_output= False)),
                    ('scaler' , StandardScaler())
                ])

preprocessor=ColumnTransformer(
    [
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)

]
)

In [104]:
X_train , X_test, y_train , y_test = train_test_split(X,y , test_size = 0.2 , random_state=40)

In [105]:
X_train

,total_bill,tip,sex,smoker,day,size
26,13.37,2.00,Male,No,Sat,2
167,31.71,4.50,Male,No,Sun,4
222,8.58,1.92,Male,Yes,Fri,1
121,13.42,1.68,Female,No,Thur,2
229,22.12,2.88,Female,Yes,Sat,2
...,...,...,...,...,...,...
184,40.55,3.00,Male,Yes,Sun,2
165,24.52,3.48,Male,No,Sun,3
7,26.88,3.12,Male,No,Sun,4
219,30.14,3.09,Female,Yes,Sat,4


In [106]:
X_train = preprocessor.fit_transform(X_train)
X_test =preprocessor.transform(X_test)

In [107]:
X_train

array([[-0.74771665, -0.74116827, -0.57695238, ...,  1.32154412,
        -0.69896325, -0.5318785 ],
       [ 1.38363606,  1.18545822,  1.4501776 , ..., -0.75669059,
         1.43069039, -0.5318785 ],
       [-1.30437856, -0.80282032, -1.59051737, ..., -0.75669059,
        -0.69896325, -0.5318785 ],
       ...,
       [ 0.82232562,  0.1219604 ,  1.4501776 , ..., -0.75669059,
         1.43069039, -0.5318785 ],
       [ 1.20118112,  0.09884088,  1.4501776 , ...,  1.32154412,
        -0.69896325, -0.5318785 ],
       [-0.90460466, -0.76428779, -0.57695238, ...,  1.32154412,
        -0.69896325, -0.5318785 ]])

In [108]:
X_test

array([[-1.78968549e-02,  2.94823255e-02, -5.76952379e-01,
         1.41421356e+00, -1.41421356e+00,  7.73537753e-01,
        -7.73537753e-01, -3.09039832e-01,  1.32154412e+00,
        -6.98963245e-01, -5.31878498e-01],
       [-1.35272332e-01, -1.23438466e+00,  4.36612611e-01,
         1.41421356e+00, -1.41421356e+00,  7.73537753e-01,
        -7.73537753e-01, -3.09039832e-01, -7.56690587e-01,
        -6.98963245e-01,  1.88012864e+00],
       [-1.08008681e+00, -1.31915622e+00, -5.76952379e-01,
        -7.07106781e-01,  7.07106781e-01,  7.73537753e-01,
        -7.73537753e-01, -3.09039832e-01,  1.32154412e+00,
        -6.98963245e-01, -5.31878498e-01],
       [-3.29348615e-01,  4.14807625e-01,  4.36612611e-01,
         1.41421356e+00, -1.41421356e+00,  7.73537753e-01,
        -7.73537753e-01, -3.09039832e-01, -7.56690587e-01,
         1.43069039e+00, -5.31878498e-01],
       [-1.00338601e+00, -1.12649357e+00, -5.76952379e-01,
         1.41421356e+00, -1.41421356e+00,  7.73537753e-01,
  

In [112]:
print(X)
print(y)

     total_bill   tip     sex smoker   day  size
0         16.99  1.01  Female     No   Sun     2
1         10.34  1.66    Male     No   Sun     3
2         21.01  3.50    Male     No   Sun     3
3         23.68  3.31    Male     No   Sun     2
4         24.59  3.61  Female     No   Sun     4
..          ...   ...     ...    ...   ...   ...
239       29.03  5.92    Male     No   Sat     3
240       27.18  2.00  Female    Yes   Sat     2
241       22.67  2.00    Male    Yes   Sat     2
242       17.82  1.75    Male     No   Sat     2
243       18.78  3.00  Female     No  Thur     2

[244 rows x 6 columns]
0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int64


In [115]:
def objective(trail,data=X_train,target=y_train):
  train_x,test_x,train_y,test_y=train_test_split(data,target,test_size=0.25,random_state=30)
  param={
      "tree_method":"gpu_hist",
      'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
      'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
      'colsample_bytree' :trail.suggest_categorical('colsample_bytree' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'subsample' :trail.suggest_categorical('subsample' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
      'n_estimators' :3000,
      'max_depth' :trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
      'random_state' :trail.suggest_categorical('random_state' , [10,20,30 ,2000 , 3454,243123]),
      'min_child_weight' :trail.suggest_int('min_child_weight' ,1,200)
      }
  Xgb_reg_model=xgb.XGBClassifier(**param)
  Xgb_reg_model.fit(train_x,train_y,eval_set=[(test_x,test_y)],verbose=True)
  pred_xgb=Xgb_reg_model.predict(test_x)


In [116]:
find_params=optuna.create_study()
find_params.optimize(objective,n_trials=10)
find_params.best_trial.params

[I 2023-05-28 15:01:45,688] A new study created in memory with name: no-name-593349a1-6481-4286-a3f2-dd7b066bd74a


[0]	validation_0-logloss:0.69315
[1]	validation_0-logloss:0.69315
[2]	validation_0-logloss:0.69315
[3]	validation_0-logloss:0.69315
[4]	validation_0-logloss:0.69315
[5]	validation_0-logloss:0.69315
[6]	validation_0-logloss:0.69315
[7]	validation_0-logloss:0.69315
[8]	validation_0-logloss:0.69315
[9]	validation_0-logloss:0.69315
[10]	validation_0-logloss:0.69315
[11]	validation_0-logloss:0.69315
[12]	validation_0-logloss:0.69315
[13]	validation_0-logloss:0.69315
[14]	validation_0-logloss:0.69315
[15]	validation_0-logloss:0.69315
[16]	validation_0-logloss:0.69315
[17]	validation_0-logloss:0.69315
[18]	validation_0-logloss:0.69315
[19]	validation_0-logloss:0.69315
[20]	validation_0-logloss:0.69315
[21]	validation_0-logloss:0.69315
[22]	validation_0-logloss:0.69315
[23]	validation_0-logloss:0.69315
[24]	validation_0-logloss:0.69315
[25]	validation_0-logloss:0.69315
[26]	validation_0-logloss:0.69315
[27]	validation_0-logloss:0.69315
[28]	validation_0-logloss:0.69315
[29]	validation_0-loglos

<ipython-input-115-bbdb7629433e>:5: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

<ipython-input-115-bbdb7629433e>:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.



[55]	validation_0-logloss:0.69315
[56]	validation_0-logloss:0.69315
[57]	validation_0-logloss:0.69315
[58]	validation_0-logloss:0.69315
[59]	validation_0-logloss:0.69315
[60]	validation_0-logloss:0.69315
[61]	validation_0-logloss:0.69315
[62]	validation_0-logloss:0.69315
[63]	validation_0-logloss:0.69315
[64]	validation_0-logloss:0.69315
[65]	validation_0-logloss:0.69315
[66]	validation_0-logloss:0.69315
[67]	validation_0-logloss:0.69315
[68]	validation_0-logloss:0.69315
[69]	validation_0-logloss:0.69315
[70]	validation_0-logloss:0.69315
[71]	validation_0-logloss:0.69315
[72]	validation_0-logloss:0.69315
[73]	validation_0-logloss:0.69315
[74]	validation_0-logloss:0.69315
[75]	validation_0-logloss:0.69315
[76]	validation_0-logloss:0.69315
[77]	validation_0-logloss:0.69315
[78]	validation_0-logloss:0.69315
[79]	validation_0-logloss:0.69315
[80]	validation_0-logloss:0.69315
[81]	validation_0-logloss:0.69315
[82]	validation_0-logloss:0.69315
[83]	validation_0-logloss:0.69315
[84]	validatio

[W 2023-05-28 15:01:55,153] Trial 0 failed with parameters: {'lambda': 0.010187960922400808, 'alpha': 0.5728844530689875, 'colsample_bytree': 0.2, 'subsample': 0.5, 'learning_rate': 0.02, 'max_depth': 5, 'random_state': 2000, 'min_child_weight': 161} because of the following error: The value None could not be cast to float..
[W 2023-05-28 15:01:55,155] Trial 0 failed with value None.


...

[0]	validation_0-logloss:0.69315
[1]	validation_0-logloss:0.69315
[2]	validation_0-logloss:0.69315
[3]	validation_0-logloss:0.69315
[4]	validation_0-logloss:0.69315
[5]	validation_0-logloss:0.69315
[6]	validation_0-logloss:0.69315
[7]	validation_0-logloss:0.69315
[8]	validation_0-logloss:0.69315
[9]	validation_0-logloss:0.69315
[10]	validation_0-logloss:0.69315
[11]	validation_0-logloss:0.69315
[12]	validation_0-logloss:0.69315
[13]	validation_0-logloss:0.69315
[14]	validation_0-logloss:0.69315
[15]	validation_0-logloss:0.69315
[16]	validation_0-logloss:0.69315
[17]	validation_0-logloss:0.69315
[18]	validation_0-logloss:0.69315
[19]	validation_0-logloss:0.69315
[20]	validation_0-logloss:0.69315
[21]	validation_0-logloss:0.69315
[22]	validation_0-logloss:0.69315
[23]	validation_0-logloss:0.69315
[24]	validation_0-logloss:0.69315
[25]	validation_0-logloss:0.69315
[26]	validation_0-logloss:0.69315
[27]	validation_0-logloss:0.69315
[28]	validation_0-logloss:0.69315
[29]	validation_0-loglos

[W 2023-05-28 15:03:02,022] Trial 7 failed with parameters: {'lambda': 0.04872980225381067, 'alpha': 0.20444133510994067, 'colsample_bytree': 1, 'subsample': 1, 'learning_rate': 0.02, 'max_depth': 6, 'random_state': 3454, 'min_child_weight': 34} because of the following error: The value None could not be cast to float..
[W 2023-05-28 15:03:02,031] Trial 7 failed with value None.


[0]	validation_0-logloss:1.65439
[1]	validation_0-logloss:30.82645
[2]	validation_0-logloss:30.82645
[3]	validation_0-logloss:30.82645
[4]	validation_0-logloss:30.82645
[5]	validation_0-logloss:30.82645
[6]	validation_0-logloss:30.82645
[7]	validation_0-logloss:30.82645
[8]	validation_0-logloss:30.82645
[9]	validation_0-logloss:30.82645
[10]	validation_0-logloss:30.82645
[11]	validation_0-logloss:30.82645
[12]	validation_0-logloss:30.82645
[13]	validation_0-logloss:30.82645
[14]	validation_0-logloss:30.82645
[15]	validation_0-logloss:30.82645
[16]	validation_0-logloss:30.82645
[17]	validation_0-logloss:30.82645
[18]	validation_0-logloss:30.82645
[19]	validation_0-logloss:30.82645
[20]	validation_0-logloss:30.82645
[21]	validation_0-logloss:30.82645
[22]	validation_0-logloss:30.82645
[23]	validation_0-logloss:30.82645
[24]	validation_0-logloss:30.82645
[25]	validation_0-logloss:30.82645
[26]	validation_0-logloss:30.82645
[27]	validation_0-logloss:30.82645
[28]	validation_0-logloss:30.82

[W 2023-05-28 15:03:22,123] Trial 9 failed with parameters: {'lambda': 0.0004134832688060719, 'alpha': 0.006666905664041873, 'colsample_bytree': 0.5, 'subsample': 0.5, 'learning_rate': 1, 'max_depth': 12, 'random_state': 30, 'min_child_weight': 60} because of the following error: The value None could not be cast to float..
[W 2023-05-28 15:03:22,125] Trial 9 failed with value None.


ValueError: ignored

In [132]:
best_params={'lambda': 0.7816338595163784,
 'alpha': 0.25683131748959687,
 'colsample_bytree': 0.1,
 'subsample': 0.9,
 'learning_rate': 15,
 'max_depth': 10,
 'random_state': 25,
 'min_child_weight': 124}

In [133]:
model=xgb.XGBClassifier(**best_params)

In [134]:
model.fit(X_train , y_train)

XGBClassifier(alpha=0.25683131748959687, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.1, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=0.7816338595163784,
              learning_rate=15, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=124, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [135]:
y_pred = model.predict(X_test)

In [136]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred , y_test)

0.5714285714285714

In [138]:
from sklearn.ensemble import RandomForestClassifier
model2=RandomForestClassifier()
model2.fit(X_train,y_train)
y_pred2=model2.predict(X_test)
accuracy_score(y_test,y_pred2)


0.9591836734693877